In [1]:
import sys
sys.path.append('../src')

import pandas as pd

from utils.data_wrangler import Wrangler

import time
from glob import glob
import shutil
import logging

from vnstock3 import Vnstock
from vnstock3.explorer.fmarket import Fund

In [2]:
benchmark_csv_dir = '../data/benchmark/dc_performance.csv'
fundamental_csv_dir = '../data/fundamental_data/'
historical_csv_dir = '../data/historical_data/'
gics_csv_dir = '../data/gics/gics.csv'
wr = Wrangler(benchmark_csv_dir, fundamental_csv_dir, historical_csv_dir, gics_csv_dir)

In [3]:
# wr.get_processed_sector_data() 
# wr.get_symbol_enough_data_field()

# wr.get_benchmark_data()
# len(wr.get_universe())
# tmp = wr.get_processed_fundamental_data()

In [4]:
tmp = wr.get_processed_sector_data()

In [7]:
tmp.ticker.nunique()

1375

In [43]:
def process_sector_data(sector_info): 
    # Get the symbol available in daily data
    files = glob(historical_csv_dir + '*.csv')
    available_daily_symbol = [file[-7:-4] for file in files]
    
    # Drop symbol whose sectors are NaN
    sector_info.dropna(subset=['ticker'], inplace=True)

    
    # Sector mask for current daily symbols
    mask_daily_universe = sector_info['ticker'].isin(available_daily_symbol)
    daily_universe = sector_info[mask_daily_universe]
    
    # Mapping Sector
    sector_mapping = {
        'Financials': ['Bảo hiểm', 'Dịch vụ tài chính', 'Tổ chức tín dụng'],
        'Information Technology': ['Chất bán dẫn và thiết bị bán dẫn', 'Công nghệ phần cứng và thiết bị', 'Phần mềm và dịch vụ'],
        'Health Care': ['Dược phẩm, Công nghệ sinh học và Khoa học thường thức', 'Dịch vụ và thiết bị chăm sóc sức khỏe'],
        'Consumer Discretionary': ['Dịch vụ tiêu dùng', 'Hàng tiêu dùng và trang trí', 'Phân phối và bán lẻ hàng lâu bền', 'Xe và linh kiện', 'Đồ gia dụng và cá nhân'],
        'Utilities': ['Dịch vụ tiện ích'],
        'Communication Services': ['Dịch vụ viễn thông', 'Truyền thông và giải trí'],
        'Materials': ['Nguyên vật liệu'],
        'Energy': ['Năng lượng'],
        'Real Estate': ['Phát triển và quản lý bất động sản'],
        'Consumer Staples': ['Thực phẩm, đồ uống và thuốc lá'],
        'Industrials': ['Dịch vụ chuyên biệt và thương mại', 'Tư liệu sản xuất', 'Vận tải']
    }
    
    # Reverse sector_mapping 
    industry_to_sector = {industry: sector for sector, industries in sector_mapping.items() for industry in industries}
    mapped_sector = daily_universe['sector'].map(industry_to_sector)

    # Add new Sector column based on sector_mapping
    daily_universe.loc[:, ['mapped sector']] = mapped_sector.values

    # Rename 'Sector' and 'Mapped Sector' columns
    daily_universe = daily_universe.rename(columns={
                         'sector': 'industry groups',
                         'mapped sector': 'sector'
                         })
    # Reset index
    daily_universe.reset_index(drop=True, inplace=True)
    
    return daily_universe


In [63]:
pd.read_csv('../data/GICS.csv')

,Symbol,Company Name,Sector,Exchange,Volume of listed
0,A32,CTCP 32,Hàng tiêu dùng và trang trí,UPCoM,6800000
1,AAA,CTCP Nhựa An Phát Xanh,Nguyên vật liệu,HOSE,296759974
2,AAH,CTCP Hợp Nhất,Năng lượng,UPCoM,117900000
3,AAM,CTCP Thủy sản MeKong,"Thực phẩm, đồ uống và thuốc lá",HOSE,12346411
4,AAS,CTCP Chứng khoán SmartInvest,Dịch vụ tài chính,UPCoM,229999521
...,...,...,...,...,...
3270,YenBai,CTCP Xây dựng Số 3 Yên Bái,NaN,OTC,0
3271,YRC,CTCP Đường sắt Yên Lào,Vận tải,OTC,1900000
3272,YSC,CTCP Hapaco Yên Sơn,Nguyên vật liệu,OTC,1100000
3273,YSVN,Công ty TNHH Chứng khoán Yuanta Việt Nam,Dịch vụ tài chính,Khác,30000000
